In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import transpose as tr, conjugate as co
from scipy.linalg import expm, svd
from scipy.sparse.linalg import eigsh, LinearOperator
import math
np.random.seed(1)

**Ex. 1: Expectation values of MPOs**

We now want to evaluate expectation values of Matrix Product Operators. We start by constructing the canonized MPS of $|\Psi_1\rangle$,  $|\Psi_2\rangle$ and a random state with bond dimension 2 and $N=30$. This is the same as Exercise Sheet 4, so the code is given to you.

In [3]:
sigma_x=np.array([[0,1],[1,0]])
sigma_y=np.array([[0+0j,0-1j],[0+0j,0+1j]])
sigma_z=np.array([[1,0],[0,-1]])

some helper functions

In [4]:
def inverse(S,d):
    """
    Helper function.
    Returns inverse of non-zero part of a diagonal matrix
    
    Parameters
    ----------
    S: array [d2xd2]
       S=np.diag([lambda_1, ...0,..lambda_d,0..]) diagonal, with dimension d2>=d
    d: int
       number of non-zero diagonal elements of S
       
    Returns
    -------
    array [dxd], Sinv=np.diag([1/lambda_1,...1/lambda_d]) with dimension d
    """
    d2=np.shape(S)[0]
    Sinv=np.zeros((d,d))
    for i in range(d2):
        if (S[i]>1e-3):
            Sinv[i,i]=1.0/S[i]
    return Sinv

def dot(A,B):
    """ 
    Helper function.
    Does the dot product like np.dot, but preserves the shapes also for singleton dimenstions 
    Example: If np.shape(A)=(1,3) and np.shape(B)=(3,4), 
             then dot(A,B) yields an array with shape (1,4) instead of (4)
             
    Parameters
    ----------
    A: array [nxm]
    B: array [mxs]
    
    Returns
    -------
    array [nxs], matrix multiplication of A and B
    """
    s1 = A.shape
    s2 = B.shape
    return np.dot(A,B).reshape((s1[0],s2[1]))

to obtain the Vidal canonical form

In [5]:
def right_canonize_step(M_im1,Mtilde_i,return_S = False):
    """ 
    One step of the right-normalization procedure.
    Right normalizes Mtilde_i into B matrix by performing svd, M_im1 looses its canonization 
    
    Parameters
    ----------
    Mtilde_i: array, shape (s,da,db)
              obtained from previous canonization step
              physical index: s
              left bond dim: da, right bond dim: db
              
    M_im1: array, shape (s,dleft,da)
              Tensor M^sigma_{i-1} from the original MPS representation
              physical index: s
              left bond dim: dleft, right bond dim: da
              
    return_S: if True, returns also the singular values
           
    Returns
    -------
    Mtilde_im1: array, shape (s,dleft,da)
              to be used as input in next call of right_canonize_step
    B_i: array, shape (s,da,db)
              right-normalized tensor
    if return_S: 
       S: array, shape (da) (or svd rank)
              singular values from svd decomposition of Mtilde_i
    """
    s, da, db = Mtilde_i.shape
    U, S, Vh = svd(Mtilde_i.transpose((1,0,2)).reshape((da,s*db)))
    B_i     = Vh.reshape((Vh.shape[0],s,db)).transpose((1,0,2))[:,:da,:]
    M_im1     = np.tensordot(M_im1,dot(U[:,:min(da,np.shape(S)[0])],np.diag(S[:min(da,np.shape(S)[0])])),axes=((2),(0)))
    if return_S:
        return M_im1, B_i, S
    else:
        return M_im1, B_i
    

def right_canonize_complete(M):
    """ performs right-canonization and returns right-normalized MPS [B_1, ...B_N]
    
    Parameters
    ----------
    M: list of tensors [M_1, ...M_N]
       where M_i is an array of shape (s,dleft,dright)-> s=2 (physical index), dleft and dright are the bond dimensions
       Matrix Product representation of a given state with N spins
   
    Returns
    -------
    B: list of tensors [B_1, ...B_N]
       right-normalized MPS representation of given state
    """
    N=len(M)
    B=[]
    Mitilde,Bi,Si=right_canonize_step(M[N-2],M[N-1],True)
    Bi.reshape(np.shape(Bi)[0],np.shape(Bi)[1],1)
    B.insert(0,Bi)
    for i in range(N-2):
        Mitilde,Bi,Si=right_canonize_step(M[N-3-i],Mitilde,True)
        B.insert(0,Bi)
    _,Bi,Si=right_canonize_step(np.zeros((np.shape(M[0])[0],1,1)),Mitilde,True)
    B.insert(0,Bi)
    
    return B

In [6]:
def canonize_start(B1,B2):
    """
    performs svd on first site
    
    Parameters
    ----------
    B1: array, shape (s,da,db) with da=1
        first tensor from the left of the right-canonized MPS
    B2: array, shape (s,db,dright)
        second tensor from the left of the right-canonized MPS
        
    Returns
    -------
    Gamma_1: array, shape (s,da,db)
         Vidal form tensor on first site
    Btilde_2: array, shape (s,db,dright)
        Tensor to be used as input for the next canonization step
    S: array, shape (min(da,db)) (singular values)
        np.diag(S) corresponds to Lambda_1, Vidal form tensor between site 1 and 2
    """
    s,da,db=np.shape(B1)
    reshapedB=np.reshape(B1,(s*da,db))
    U,S,Vdag=np.linalg.svd(reshapedB,full_matrices=0)
    A2=np.reshape(U,(s,da,U.shape[1]))
    Gamma_1=np.zeros((s,da,db))
    Gamma_1[:,:,:U.shape[1]]=A2
    Btilde_2=np.tensordot(np.dot(np.diag(S),Vdag),B2,axes=(1,1))
    Btilde_2=np.transpose(Btilde_2,(1,0,2)) 
    return Gamma_1,Btilde_2,S

def canonize_step(Btilde_i,S1,B_ip1):
    """
    performs svd on site i
    
    Parameters
    ----------
    Btilde_i: array, shape (s,da,db) 
        tensor on site i obtained from svd of the last step
    S1: array, 
        singular values obtained in previous step
    B_ip1: array, shape (s,db,dright)
        tensor on site i+1 of the right-canonized MPS
        
    Returns
    -------
    Gamma_i: array, shape (s,da,db)
        Gamma_i: Vidal form tensor on site i
    Btilde_ip1: array, shape (s,db,dright)
        Tensor to be used as input for the next canonization step
    S2: array, shape (min(da,db)) (singular values)
        np.diag(S) corresponds to Lambda_i, Vidal form tensor between site i and i+1
        to be used as input for next step
    """
    s,da,db=np.shape(Btilde_i)
    reshapedB=np.reshape(Btilde_i,(s*da,db))
    U,S2,Vdag=np.linalg.svd(reshapedB,full_matrices=0)
    Gamma_i=np.reshape(U,(s,da,U.shape[1]))[:,:,:db]
    Gamma_i=np.tensordot(inverse(S1,da),Gamma_i,axes=(1,1))
    Gamma_i=np.transpose(Gamma_i,(1,0,2))
    Btilde_ip1=np.tensordot(np.dot(np.diag(S2),Vdag),B_ip1,axes=(1,1))
    Btilde_ip1=np.transpose(Btilde_ip1,(1,0,2))  
    return Gamma_i,Btilde_ip1,S2

def canonize_end(Btilde_N,S1):
    """
    performs svd on last site
    input: M1 obtained from svd of the last step, S1 singular values of svd of last step
    output: Gamma matrix of last site
    wave-function is normalized by setting lambda of last site to 1
    
    performs svd on last site
    
    Parameters
    ----------
    Btilde_N: array, shape (s,da,db) 
        tensor on site i obtained from svd of the last step
    S1: array, 
        singular values obtained in previous step

    Returns
    -------
    Gamma_N: array, shape (s,da,db) with db=1
        Gamma_N: Vidal form tensor on site N
   
    The wave-function is normalized by setting lambda of last site to 1
    """
    s,da,db=np.shape(Btilde_N)
    reshapedB=np.reshape(Btilde_N,(s*da,db))
    U,S2,Vdag=np.linalg.svd(reshapedB,full_matrices=0)
    Gamma_N=np.reshape(U,(s,da,U.shape[1]))[:,:,:1]
    Gamma_N=np.tensordot(inverse(S1,da),Gamma_N,axes=(1,1))
    Gamma_N=np.transpose(Gamma_N,(1,0,2))
    return Gamma_N


In [7]:
def canonize(M):
    """
    Given an MPS, this function computes the Vidal form 
    by first right-normalizing and then performing a sweep from the left.
    
    Parameters
    ----------
    M: list of tensors [M_1, ...M_N]
       where M_i is an array of shape (s,dleft,dright)-> s=2 (physical index), dleft and dright are the bond dimensions
       Matrix Product representation of a given state with N spins
    
    Returns
    -------
    Gammas: list
       [Gamma1,...GammaN]
    Lambdas: list
       [Lambda1,....LambdaN-1]
           
    s.t. the MPS in Vidalform is: [Gamma1,Lambda1,Gamma2,....LambdaN-1,GammaN]
    """
    N=len(M)
    M=right_canonize_complete(M)
    Gammas=[]
    Lambdas=[]
    Gamma1,M_i,S_i=canonize_start(M[0],M[1])
    Gammas.append(Gamma1)
    Lambdas.append(S_i)
    for i in range(N-2):
        Gammai,M_i,S_i=canonize_step(M_i,S_i,M[i+2])
        Gammas.append(Gammai)
        Lambdas.append(S_i)
    Gamma_N=canonize_end(M_i,S_i)
    Gammas.append(Gamma_N)
    
    return Gammas,Lambdas

getting the MPS for $|\Psi_1\rangle$, $|\Psi_2\rangle$ and a random state

In [8]:
def get_MPsi1(N):
    """ 
    returns the MPS representation M_AFGHZ=[A1,...AN] of Psi_1, where Ai has the dimensions [s,da,db], s is the spin degree of freedom
    """
    A1=np.zeros((2,1,2))
    A1[0,0,:]=1.0/np.sqrt(2)*np.array([[1,0]])
    A1[1,0,:]=1.0/np.sqrt(2)*np.array([[0,1]])
    #print A1
 
    Ai_odd= np.zeros((2,2,2))
    Ai_odd[0,:,:]=np.array([[0,0],[0,1]])
    Ai_odd[1,:,:]=np.array([[1,0],[0,0]]) 
    
    Ai_even=np.zeros((2,2,2))
    Ai_even[0,:,:]=np.array([[1,0],[0,0]])
    Ai_even[1,:,:]=np.array([[0,0],[0,1]])

    # print A2

    A_final_odd=np.zeros((2,2,1))
    A_final_odd[0,:,0]=np.array([1,0])
    A_final_odd[1,:,0]=np.array([0,1])

    A_final_even=np.zeros((2,2,1))
    A_final_even[0,:,0]=np.array([0,1])
    A_final_even[1,:,0]=np.array([1,0])

    M_AFGHZ=[A1]
    for i in range(N-2):
        if ((i+1)%2==0):
            M_AFGHZ.append(Ai_even)
        elif ((i+1)%2==1):
            M_AFGHZ.append(Ai_odd)
    if (N%2==0):
        M_AFGHZ.append(A_final_even)
    else:
        M_AFGHZ.append(A_final_odd)
    return M_AFGHZ

In [9]:
def get_MPsi2(N):
    """ 
    returns the MPS representation M_x=[A1,...AN] of Psi_2, where Ai has the dimensions [s,da,db], s is the spin degree of freedom
    """
    A1=np.zeros((2,1,1))
    A1[0,0,:]=1.0/np.sqrt(2)
    A1[1,0,:]=1.0/np.sqrt(2)
    Ai=np.zeros((2,1,1))
    Ai[0,:,:]=1.0/np.sqrt(2)
    Ai[1,:,:]=1.0/np.sqrt(2)
    A3=np.zeros((2,1,1))
    A3[0,:,0]=1.0/np.sqrt(2)
    A3[1,:,0]=1.0/np.sqrt(2)
    M_x=[A1]
    for i in range(N-2):
        M_x.append(Ai)
    M_x.append(A3)
    return M_x

In [10]:
def make_randomMPS(d,N):
    """
    returns random MPS with bond dimension d, N spins
    """
    A1=np.random.rand(2,1,d)
    Ai=np.random.rand(2,d,d)
    A3=np.random.rand(2,d,1)
    M_GHZ=[A1]
    for i in range(N-2):
        M_GHZ.append(Ai)
    M_GHZ.append(A3)
    return M_GHZ

In [11]:
Gammas_rand,Lambdas_rand=canonize(make_randomMPS(2,30))

M_Psi1=get_MPsi1(30)
M_Psi2=get_MPsi2(30)

Psi1_Gammas,Psi1_Lambdas=canonize(M_Psi1)
Psi2_Gammas,Psi2_Lambdas=canonize(M_Psi2)

Voila! We finally have the results from last week: $|\Psi_1\rangle$, $|\Psi_2\rangle$ and a random MPS (all canonized).

### Exercise 1.2
Next, we construct an MPO representation for the transverse field Ising model

In [12]:
def make_Ising(N, h=0):
    """ return MPO [O1, ...ON] of the transverse field Ising model with dim(Oi)=[s,s,da,db], here s=2 and da=db=3"""
    #implement the MPO representation of the transverse field Ising model here
    O1 = np.zeros((2,2,1,3))+0j #frist two are the matrizes of our operaiotr, the one is the linezeile and the 3 is the spaltenzeile
    O1[:,:,0,0] = -1*h*sigma_x
    O1[:,:,0,1] = sigma_z
    O1[:,:,0,2] = np.eye(2)
    
    O2 = np.zeros((2,2,3,3))+0j
    O2[:,:,0,0] = np.eye(2)
    O2[:,:,1,0] = sigma_z
    O2[:,:,2,0] = -h*sigma_x
    O2[:,:,2,1] = sigma_z
    O2[:,:,2,2] = np.eye(2)
    
    O3 = np.zeros((2,2,3,1))+0j
    O3[:,:,0,0] = np.eye(2)
    O3[:,:,1,0] = sigma_z
    O3[:,:,2,0] = -h*sigma_x
    
    O_Ising = [O1]
    for i in range(N-2):
        O_Ising.append(O2)
    O_Ising.append(O3)

    return O_Ising

We implement the contraction $\langle \Psi|H|\Psi\rangle$ piecewise, similarly to last week.

In [ ]:
#5.2.5 Computing expecation values, Fromulas (5.22) to (5.25)

def begin_exp(G1, Lam1,H):
    """
    Performs first step of computing the expectation value <Psi|H|Psi>
    Parameters
    ----------
    G1: array, shape (2,1,db_psi)
        Gamma_1 of Psi (Vidal tensor at site 1)

    Lam1: array, shape (db_psi) (careful with svd rank, if truncated)
        Lambda_1 of Psi (Vidal tensor between sites 1 and 2)

    H: array, shape (s,s,1,db_H)
        MPO at site 1

    Returns
    -------
    L: array, shape (db_psi, db_H, db_psi)
       contraction at site 1, including the Lambda matrices between site 1 and 2
       to be used as input for next step
    """

    A1=np.tensordot(G1[:,:,:np.shape(Lam1)[0]],np.diag(Lam1),axes=(2,0)) #np.shape(Lam1)[0] is the svd rank where we trunkated. Look at the comments above to find out which axes
   
    Adag1=np.conj(A1) #create the complex conjugate of A1 for Formula (5.22)
                                                                                    ##########Why first the A1 and after Adag1?##########
    L=np.tensordot(A1,H,axes=(0,0)) #contract A1 with H, the result is a 3D tensor. Look at the comments above to find out which axes
    L=np.tensordot(L,Adag1,axes=(2,0)) #bc of complec condjugare now axes 2 and 0
    L=np.reshape(L,(np.shape(A1)[2],np.shape(H)[3],np.shape(Adag1)[2])) #reshape the tensor to the shape given by the comment above
    return L

def step_exp(L,G1,Lam1,H):
    """
    Performs i'th step of computing the expectation value <Psi|H|Psi>
    Parameters
    ----------
    L: array, shape (da_psi, da_H, da_psi)
        contraction obtained in previous step

    G1: array, shape (2,da_psi,db_psi)
        Gamma_i of Psi (Vidal tensor at site i)

    Lam1: array, shape (db_psi) (careful with svd rank, if truncated)
        Lambda_i of Psi (Vidal tensor between sites i and i+1)

    H: array, shape (s,s,da_H,db_H)
        MPO at site i

    Returns
    -------
    L: array, shape (db_psi, db_H, db_psi)
       contraction up to site i, including the Lambda matrices between site i and i+1
       to be used as input for next step
    """
    A1=np.tensordot(G1[:,:,:np.shape(Lam1)[0]],np.diag(Lam1),axes=(2,0))
    Adag1=np.conj(A1)

    L=np.tensordot(L,A1,axes=(0,1)) #now that we work with a matrix we need to use axes 0 and 1
    L=np.tensordot(L,H,axes=([0,2],[2,0])) #contract L with H, the result is a 3D tensor. Look at the comments above to find out which axes
    L=np.tensordot(L,Adag1,axes=([0,2],[1,0])) #double contrnaction with Adag1, axes 0 and 2
    return L

def end_exp(L,G1,H):
    """
    G1: Gamma matrices of Psi at site N
    H: MPO at site N
    L: contraction up to site N-1
    returns complete contraction

    Performs the las step of computing the expectation value <Psi|H|Psi>

    Parameters
    ----------
    L: array, shape (da_psi, da_H, da_psi)
       contraction obtained in previous step

    G1: array, shape (2,da_psi,1)
       Gamma_N of Psi (Vidal tensor at site N)

    H: array, shape (s,s,da_H,1)
       MPO at site N

    Returns
    -------
    exp_value: real or complex
       contraction up to site N, corresponding to the expectation value <Psi|H|Psi>
    """
    A1=G1
    Adag1=np.conj(A1)
    L=np.tensordot(L,A1,axes=(0,1))
    #print(np.shape(L))
    #print(np.shape(H))
    L=np.tensordot(L,H,axes=([0,2],[2,0]))                                              ##########Why this axes?##########
    L=np.tensordot(L,Adag1,axes=([0,2],[1,0]))
    exp_value=L[0,0,0]
    return exp_value

def calculate_exp(Gammas1,Lambdas1,Hamiltonian):
    """returns the expectation value <Psi|H|Psi>
    with Gammas1,Lambdas1 the canonized MPS representation of Psi
    and Hamiltonian an MPO representation of H
    """
    L=begin_exp(Gammas1[0],Lambdas1[0],Hamiltonian[0])
    for i in range(len(Gammas1)-2):
        L=step_exp(L,Gammas1[i+1],Lambdas1[i+1],Hamiltonian[i+1])
    L=end_exp(L,Gammas1[-1],Hamiltonian[-1])
    return L

Now calculate the expectation values (you can verify this analytically!)

In [21]:
h_values=[0,1,2]
for h in h_values:
    O_Ising=make_Ising(30,h)
    print("The expectation value for Psi1, h:",h, "is equal to:",calculate_exp(Psi1_Gammas,Psi1_Lambdas,O_Ising))
    print("The expectation value for Psi2, h:",h, "is equal to:",calculate_exp(Psi2_Gammas,Psi2_Lambdas,O_Ising))
    print("The expectation value for a randomly generated MPS, h:",h, "is equal to:",calculate_exp(Gammas_rand,Lambdas_rand,O_Ising))

The expectation value for Psi1, h: 0 is equal to: (-28.999999999999996+0j)
The expectation value for Psi2, h: 0 is equal to: 0j
The expectation value for a randomly generated MPS, h: 0 is equal to: (13.29143396042737+0j)
The expectation value for Psi1, h: 1 is equal to: (-28.999999999999996+0j)
The expectation value for Psi2, h: 1 is equal to: (-29.999999999999858+0j)
The expectation value for a randomly generated MPS, h: 1 is equal to: (-7.457348864190328+0j)
The expectation value for Psi1, h: 2 is equal to: (-28.999999999999996+0j)
The expectation value for Psi2, h: 2 is equal to: (-59.999999999999716+0j)
The expectation value for a randomly generated MPS, h: 2 is equal to: (-28.206131688808046+0j)


In [15]:
ten=np.zeros((2,10))